In [1]:
import pandas as pd
import numpy as np

## General Components of XGBoost

- Gradient Boost
- Regularization
- Unique regression trees (XGBoost Trees)
- Approximate greedy algorithm
- Weighted quantile sketch
- Sparsity-awar split finding
- Parallel learning
- Cache-aware access
- Blocks for out-of-core computation

Many of these features are optimizations that make XGBoost computationally efficient. The most significant and most fundamental difference between XGBoost and traditional gradient-boosting is its unique approach to regression trees that it iteratively fits upon the pseudo-residuals (gradients).

# XGBoost Trees

## Regression Context

#### Similarity Scores and Gain

Like with traditional GBM, XGBoost begins with an intial prediction, then iteratively fits trees upon the gradients (pseudo-residuals), beginning with this initial prediction. Unlike traditional GBM, XGBoost does not use ordinary regression trees to do iterative fitting, but its own variant of regression trees.

Each tree starts with a *single leaf* comprised of **all of the residuals*.\
Then, before any splitting is done, a ***Similarity Score*** is computed for the residuals. This is simply:
$$\text{Similarity Score} = \frac{(\text{Sum of Residuals})^2}{\text{Number of Residuals} + \lambda}$$
Where $\lambda$ is a regularization parameter.

The splitting criteria is now the **maximization** of the similarity score. Thus, the objective of the XGBoost tree is (loosely) to partition the predictor space such that pseudo-residuals are well-clustered.\
The quality of a split is measured by its ***Gain*** which is defined as the sum of the similarity scores for the two child nodes minus the similarity score for the parent node, i.e.:
$$\text{Gain} = \text{Left}_{Similarity}  + \text{Right}_{Similarity} - \text{Parent}_{Similarity}$$
Splits are chosen to maximize this gain.

#### Pruning

Splits of XGBoost trees are ***Pruned*** based on their similarity score and gain values.\
A pruning parameter is used to prune splits, this parameter is denoted $\gamma$.\
Pruning is conducted recursively, that is, beginning with the last split. If the gain for a split is less than $\gamma$, then the branch (split) is removed. If the gain is greater than $\gamma$, then it is not removed and pruning stops.\
To summarize the pruning decision, with decision function $f(\cdot)$:
$$f(\text{Gain}) = \begin{cases} 1, \ \text{Gain} < \gamma \\ 0, \ \text{Gain} > \gamma \end{cases}$$

Note that depending on the value of the pruning parameter $\gamma$, it is possible for the *entire tree* to be pruned. This would effectively mean that the boosting process is stopped and may thus be used as an early stopping criterion.

#### Regularization

The similarity scores include a regularization parameter $\lambda$. This parameter is used to reduce the similarity score's sensitivity to individual observations with large residuals. This is effective for reducing the influence of observations with large residuals because it increases the denominator which marginally decreases the similarity scores for nodes with fewer residuals than for nodes with more residuals. And since the gain is maximized by making splits that isolate large residuals, the large residuals will effectively have penalized similarity scores relative to the smaller residuals.

In other words, the decrease in similarity scores imposed by $\lambda$ is **inversely proportional** to the number of residuals in a node, which in turn is inversely proportional to the size of the residuals in that node. Thus, the penalty imposed by $\lambda$ increases as the size of the residuals increase, *relative to* the other residuals in the data set.

**Note:** It is possible for regularization via $\lambda > 0$ to produce splits with **negative** gains. For example, when a split separates two residuals, then the sum of their regularized similarities may be less than the regularized similarity of the parent node, in which case the gain would be negative. In such cases, even when the pruning parameter is $0$ (i.e. $\gamma = 0$), the branch *will be pruned* because the negative gain is less than $\gamma$.\
So, when regularization is used for similarity scores, pruning may still occur even when $\gamma = 0$

#### Output Values

Output values from each terminal node of an XGBoost tree are computed as:
$$\text{Output Value} = \frac{\text{Sum of Residuals}}{\text{Number of Residuals} + \lambda}$$
Note that the regularization parameter $\lambda$ is included in the output value, and that the only difference between the output values and the similarity scores is that the output values do not square the sum of the residuals in the numerator.

#### Update

Predictions are updated by each iterative XGBoost tree by adding the new output values weighted by a learning rate parameter $\epsilon$ to the previous predictions.

## Classification Context
The following is for a *binary classification*

#### Similarity Scores

Like with the regression context, we begin with an initial prediction then make splits so as to maximize the within-node similarity scores of the residuals drawn from this initial prediction. However, in the classification context, the similarity score computation is different. It is:
$$\text{Similarity Score} = \frac{\big(\sum r_i\big)^2}{\sum\big(p_i (1 - p_i) \big) + \lambda}$$
Where $r_i$ is the residual for observation $i$ and $p_i$ is the probability for observation $i$ predicted by the previous iteration of the algorithm (at $m=1$ this is the initial prediction).

**Terminology**: The denomiator of the similarity score in XGBoost (excluding $\lambda$) is called ***Cover***. Implementations of XGBoost treat cover as a tunable value by allowing you to set a minimum value for the cover of a node such that splits are only made when the cover of child nodes exceeds the minimum value. This is important for the classification context because the XGBoost package set `min_child_weight=1` by default (this is the cover) and $\sum(p_i (t - p_i))$ may easily be less than $1$, which could prevent many splits from occuring.

#### Gain
Gain is computed the same as in the regression context

#### Regularization
Regularization works the same as in regression

#### Update
Predictions are updated by adding the new output values weighted by a learning rate $\epsilon$ to the previous iteration's predictions.

## XGBoost Maths Details

In the previous section, we treated the similarity scores and gains as functions of residuals. This isn't entirely true. As with traditional gradient boosting, the values we use for fitting are gradients (or pseudo-residuals) not ordinary residuals. The default loss functions used for XGBoost regression and classification have pseudo-residuals that are equivalent to the ordinary residuals, but for different loss functions this may not be the case.\
By default, regression is done using a weighted squared-error loss:
$$L(y_i, \hat{y}_i) = \frac{1}{2}(y_i - \hat{y}_i)^2$$
And by default, classification is done using cross-entropy log-loss:
$$L(y_i, p_i) = - \big[ y_i \ln(p_i) + (1 - y_i) \ln (1 - p_i) \big]$$

XGBoost builds trees by minimizing the $l_2$-regularized versions of these loss functions:
$$\sum_{i=1}^n L(y_i, p_i) + \frac{1}{2}\lambda O^2$$
Where $O$ denotes the output value for the leaf. In the XGBoost paper, the output values for a leaf are denoted $\omega$ and are referred to as scores.\
Splits are made by selecting the output value that minimizes this regularized objective function.\
The default implementation of XGBoost uses second-order Taylor approximations to solve the objective function minimization problem in all instances (i.e. for regression and for classification for all loss functions). This approximation is:
$$L(y_i, \hat{y}_i + O) \approx L(y_i, \hat{y}_i) + \big[\frac{d}{d\hat{y}_i}L(y_i, \hat{y}_i)\big]O + \frac{1}{2} \big[ \frac{d^2}{d\hat{y}_i^2}L(y_i, \hat{y}_i)\big]O^2$$
In the XGBoost paper and documentation, the notation is simplified by expressing the first derivative as $g$ (since it is a gradient in vectorized computation) and the second derivative as $h$ (since it is a Hessian in vectorized computation).\
So, the documentation expressed this as:
$$L(y_i, \hat{y}_i + O) = L(y_i, \hat{y}_i) + g_iO + \frac{1}{2}h_iO^2$$

Now, the objective function is a sum of loss functions, so we want to select $O$ such that:
$$O = \argmin_O \sum_{i=1}^n \big[ L(y_i, \hat{y}_i) + gO + \frac{1}{2}hO^2 \big] + \frac{1}{2}\lambda O^2$$
Noting that the loss functions, gradients, and hessians are all independent of $O$, we may solve for $O$ as:
$$
\frac{d}{dO} \bigg[ \big(\sum_{i=1}^n g_i\big)O + \frac{1}{2}\big(\sum_{i=1}^n h_i + \lambda \big)O^2 \bigg] = 0 \\ \ \\
\implies \big(\sum_{i=1}^n g_i\big) + \big(\sum_{i=1}^n h_i + \lambda \big)O = 0 \\ \ \\
\implies O = - \frac{\sum_{i=1}^n g_i}{\sum_{i=1}^n h_i + \lambda}
$$


This optimal value for the leaf output value yields the exact output expressions that we used in the first part of this notebook for regression and classification with the weighted squared error loss and log-loss respectively.

This is a simple and easily generalizable (i.e suitable for many loss-functions) objective function that works analogously to the impurity measures used by decision trees but is far more flexible.

Now for the similarity scores:\
The similarity scores are computed using only the simplified objective function (i.e. excluding summands that are independent of $O$). Denoting the optimal output value for the leaf $O^*$, this is:
$$O^*\sum g_i - O^{*2}\frac{1}{2}(\sum h_i + \lambda)$$
Which simplifies to:
$$\frac{1}{2}\frac{(\sum g_i)^2}{\sum h_i + \lambda}$$
This gives the similarity scores used in the first part of this notebook.

**NOTE**: The regularization parameter $\lambda$ literally is the $l_2$-regularization parameter. Like with $l_2$ regularization in linear models, it prevents overfitting by penalizing high-leverage points. However, the way that it penalizes splits is quite different from how it penalizes predictors in linear models.